In [1]:
import os, sys, csv, json, re
import numpy as np
import pandas as pd

In [6]:
import datetime as dt
import psychropy
import ashrae_clear_sky as acs

station_name = 'KLAX'
YEAR = '2019'
START_MON = '1'
START_DAY = '1'
END_MON = '2'
END_DAY = '28'

# Input and output file location
input_folder = 'station-result'
output_folder = 'epw-results'
location = 'Los_Angeles'
LOC_WEATHER_FILE = 'epw-raw/USA_CA_Los.Angeles.Intl.AP.722950_TMY3.epw'

CSV_FILE = os.path.join(input_folder, station_name + '-' + str(YEAR) + '.csv')
EPW_FILE = os.path.join(output_folder, station_name + '-' + location + '-' + str(YEAR) + '.epw')

# to be modified according to the prepared CSV file format
csv_headers_epw_map = {6: 'dry_bulb', 7:'dew_point', 8:'rel_humi', 9:'pressure', 
                       13: 'global_hori', 14: 'dir_norm', 15: 'dif_hori', 20:'wind_dir', 21:'wind_speed'}

days_in_mon = {1:31, 2:28, 3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31}
cloud_cover_header = 'cloud_layer_1_digit_m'

In [7]:
df_csv_result = pd.read_csv(CSV_FILE, sep=',', index_col=0, parse_dates=True, infer_datetime_format=True, encoding='UTF-8')

In [8]:
df_csv_result.head()

,date_time,dry_bulb,rel_humi,dew_point,pressure,wind_speed,wind_dir,wind_chill,heat_index,cloud_layer_1,cloud_layer_1_digit_m
0,2019-01-01-00:00,10.757143,19.727143,-11.630000,101073.917143,0.845000,45.000000,NaN,NaN,24.571429,6.0
1,2019-01-01-01:00,10.238462,20.241538,-11.750000,101147.085385,1.661538,168.461538,NaN,NaN,188.615385,6.0
2,2019-01-01-02:00,9.723077,19.960000,-12.330769,101165.236923,1.818462,209.230769,NaN,NaN,115.923077,6.0
3,2019-01-01-03:00,7.930769,27.346154,-10.195385,101167.830000,1.898462,157.692308,NaN,NaN,116.000000,6.0
4,2019-01-01-04:00,7.446154,29.906154,-9.263077,101217.099231,0.395385,29.230769,NaN,NaN,139.000000,6.0


In [9]:
file_name = os.path.join('epw-raw', 'solar-parameters-la.csv')
df_solar = pd.read_csv(file_name, sep=',', parse_dates=True, infer_datetime_format=True, encoding='UTF-8')
df_solar.describe()
for field in ['A', 'B', 'C', 'AVSC', 'cos-zenith']:
    df_csv_result[field] = df_solar[field]

In [13]:
# write EPW
data_to_write = dict()
for i, row in df_csv_result.iterrows():
#     row['hum_rat'] = psychropy.Hum_rat2(float(row['dry_bulb']), float(row['hurs']), float(row['ps']))
#     row['dew'] = psychropy.Dew_point(float(row['ps']), float(row['hum_rat']))
    mon = int(row['date_time'].split('-')[1])
    day = int(row['date_time'].split('-')[2])
    hour = int(row['date_time'].split('-')[3].split(':')[0])
    row['year'] = row['date_time'].split('-')[0]
    data_to_write[str(mon) + "/" + str(day) + "/" + str(hour)] = row

In [14]:
data_to_write['1/1/0']

date_time                2019-01-01-00:00
dry_bulb                          10.7571
rel_humi                          19.7271
dew_point                          -11.63
pressure                           101074
wind_speed                          0.845
wind_dir                               45
wind_chill                            NaN
heat_index                            NaN
cloud_layer_1                     24.5714
cloud_layer_1_digit_m                   6
A                                 1232.83
B                                0.141412
C                               0.0567172
AVSC                              1.03348
cos-zenith                      -0.974462
year                                 2019
Name: 0, dtype: object

In [15]:
import math
f = open(EPW_FILE, 'w')

with open(LOC_WEATHER_FILE, newline='') as csvfile:
    csvreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in csvreader:
        if (len(row) == 35):
            mon = row[1]
            day = row[2]
            hour = row[3]
            minute = row[4]
            # parse 8/1/18 0:00 to 7/31/18 24:00
            if int(hour) == 24:
                hour = 0
                if int(day) == days_in_mon[int(mon)]:
                    mon = int(mon) + 1
                    day = 1
                else:
                    day = int(day) + 1

            # Notes: The format of the CSV time stamp
            cur_time_stamp = str(mon) + "/" + str(day) + "/" + str(hour)
            if cur_time_stamp in data_to_write:
                cur_data_to_write = data_to_write[cur_time_stamp]
                row[0] = '2021'
                row[0] = str(cur_data_to_write['year'])
                # calculate solar items using clout cover data
                if cloud_cover_header in cur_data_to_write:
                    if np.isnan(cur_data_to_write[cloud_cover_header]):
                        cloud_cover_percent = int(row[23]) / 10.0
                    else:
                        cloud_cover_percent = (cur_data_to_write[cloud_cover_header] - 1) / 9.0
                    cloud_cover_tenth = int(cloud_cover_percent * 10)
                    (row[14], row[15]) = acs.ashrae_clear_sky(cloud_cover_percent, 
                                                              cur_data_to_write['cos-zenith'],
                                                              cur_data_to_write['A'],
                                                              cur_data_to_write['B'],
                                                              cur_data_to_write['C'],
                                                              cur_data_to_write['AVSC'])
                    row[12] = '9999'
                    row[23] = str(cloud_cover_tenth)
#                 if 'is_rain' in cur_data_to_write:
#                     if cur_data_to_write['is_rain'] == 1:
#                         row[26] = '0'
#                         row[27] = '919999999'
                for epw_col in csv_headers_epw_map:
                    if csv_headers_epw_map[epw_col] in cur_data_to_write:
                        try:
                            row[epw_col] = str(round(cur_data_to_write[csv_headers_epw_map[epw_col]], 3))
                        except:
                            row[epw_col] = str(cur_data_to_write[csv_headers_epw_map[epw_col]])
            else:
                print(cur_time_stamp + ' not found.')
        f.write(','.join(row) + '\n')

f.close()

13/1/0 not found.
